In [258]:
import numpy as np # 다양한 연산을 위한 라이브러리
from PIL import Image # 이미지 파일을 가져오기 위한 라이브러리

gaussian kernel 생성 함수

In [259]:
def gaussian_kernel(sigma, size=9): # size*size 가우시안 커널 생성
    kernel = np.fromfunction( lambda x, y: (1/ (2 * np.pi * sigma**2)) * np.exp(- ((x - (size-1)/2)**2 + (y - (size-1)/2)**2) / (2*sigma**2)), (size, size) )
    # x: 가우시안
    # y: 가우시안 커널 가로 세로 사이즈
    return kernel / np.sum(kernel) # 생성된 커널을 전체 원소의 합으로 나누어 정규화

laplacian kernel 생성 함수

In [260]:
def laplacian_kernel():
    kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
    return kernel

padding

In [261]:
def zero_padding(matrix, pad_size):
    m, n = matrix.shape # 입력 행렬의 크기를 가져온다. m과 n은 각각 행렬의 세로, 가로 크기이다.
    padded_matrix = np.zeros((m + 2 * pad_size, n + 2 * pad_size), dtype=matrix.dtype) # 패딩 크기만큼 행과 열을 각각 양쪽으로 늘림
    padded_matrix[pad_size:-pad_size, pad_size:-pad_size] = matrix # 원본 행렬을 새로운 행렬의 중앙에 배치
    return padded_matrix # 제로 패딩이 적용된 행렬 반환

**이미지에 필터를 적용하는 함수**

In [262]:
def convolution(image, kernel):
    km, kn = kernel.shape # 커널의 크기 가져오기. km과 kn은 각각 커널의 가로, 세로 크기
    km2, kn2 = km // 2, kn // 2 # 커널 중심 계산
    padded_image = zero_padding(image, km2) # 제로 패딩 적용
    m, n = padded_image.shape # 패딩된 이미지의 크기 가져오기
    result = np.zeros((m-km2*2, n-kn2*2), dtype=np.float32) # 컨볼루션 연산 결과를 저장할 배열 초기화

    for i in range(km2, m-km2): # 이미지를 순회하면서 컨볼루션 연산 수행
        for j in range(kn2, n-kn2):
            temp = padded_image[i-km2:i+km2+1, j-kn2:j+kn2+1] # 커널 크기만큼의 부분 행렬 추출
            result[i-km2, j-kn2] = np.sum(temp * kernel) # 컨볼루션 연산 수행 후 결과를 결과 행렬에 저장

    return result # 컨볼루션이 적용된 결과 이미지 반환

최종 엣지 결정 함수

In [263]:
def threshold(image, thres):
    return np.where(image > thres, 255, 0)

In [264]:
# 이미지 로드
image_path = 'img.png'
image = np.array(Image.open(image_path).convert('L'))

sigma = 1.1

# 가우시안 스무딩 적용
gaussian = gaussian_kernel(sigma)
blurred_image = convolution(image, gaussian)
img = Image.fromarray(np.uint8(blurred_image), 'L')
img.save('./blurred_img.png')

# LoG 필터 적용
laplacian = laplacian_kernel() # 라플라시안 커널 생성
Log_filter = convolution(gaussian, laplacian) * -30 # LoG 필터(Laplacian Of Gaussian) 생성
result = convolution(image, Log_filter)

img = Image.fromarray(np.uint8(result), 'L')
img.save('./mid_img.png')

Log_result = threshold(result, 130) # 임계값을 넘은 픽셀에 대해서만 edge로 판정하여 남김

# Log 필터 적용 결과 이미지 저장하기
img = Image.fromarray(np.uint8(Log_result), 'L')
img.save('./Log_converted_img.png')

## Canny Edge Detector

In [265]:
def threshold2(image, thres_high, thres_low):
  m, n = image.shape
  result = np.zeros((m, n), dtype=np.float32)
  dx = [1,0,-1,0,1,-1]
  dy = [1,0,-1,0,1,-1]
  for x in range(m): # 이미지를 순회하면서 컨볼루션 연산 수행
        for y in range(n):
          # 이미지에서 thres_high을 초과하는 부분은 255로, thres_low 미만인 부분은 0으로 변환
          if (image[x,y] > thres_high) :
            result[x,y] = 255
          elif (image[x,y] < thres_low):
            result[x,y] = 0
          else :
            for n in range(1,2):
              for k in range(6):
                nx = x + dx[k]*n
                ny = y + dy[k]*n
                if -1 < nx and nx < m and -1 < ny and ny < n and image[nx,ny]>thres_high:
                  result[x,y] = 255
  return result

In [266]:
Canny_result = threshold2(result, 70, 210) # Canny

# Log 필터 적용 결과 이미지 저장하기
img = Image.fromarray(np.uint8(Canny_result), 'L')
img.save('./Canny_converted_img.png')

## Nonmax_suppression

In [267]:
def sobel_filter(image):
    kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    kernel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    gradient_x = convolve2D(image, kernel_x)
    gradient_y = convolve2D(image, kernel_y)

    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    gradient_direction = np.arctan2(gradient_y, gradient_x) * (180 / np.pi)

    return gradient_magnitude, gradient_direction

def convolve2D(image, kernel):
    m, n = image.shape
    km, kn = kernel.shape
    km2, kn2 = km // 2, kn // 2

    result = np.zeros((m - km + 1, n - kn + 1))

    for i in range(m - km + 1):
        for j in range(n - kn + 1):
            result[i, j] = np.sum(image[i:i+km, j:j+kn] * kernel)

    return result

def nonmax_suppression(gradient_magnitude, gradient_direction):
    m, n = gradient_magnitude.shape
    result = np.zeros((m, n), dtype=np.uint8)

    for i in range(1, m-1):
        for j in range(1, n-1):
            angle = gradient_direction[i, j]

            # 주변 픽셀 좌표 계산
            if (0 <= angle < 22.5) or (157.5 <= angle <= 180):
                neighbors = [gradient_magnitude[i, j-1], gradient_magnitude[i, j+1]]
            elif (22.5 <= angle < 67.5):
                neighbors = [gradient_magnitude[i-1, j+1], gradient_magnitude[i+1, j-1]]
            elif (67.5 <= angle < 112.5):
                neighbors = [gradient_magnitude[i-1, j], gradient_magnitude[i+1, j]]
            else:
                neighbors = [gradient_magnitude[i-1, j-1], gradient_magnitude[i+1, j+1]]

            # 현재 픽셀이 주변 픽셀보다 강한 엣지인지 확인
            if gradient_magnitude[i, j] >= max(neighbors):
                result[i, j] = gradient_magnitude[i, j]

    return result

gaussian = gaussian_kernel(1.0)
blurred_image = convolution(image, gaussian)

# Sobel 필터 적용
gradient_magnitude, gradient_direction = sobel_filter(blurred_image)

# Non-maximum suppression 수행
result = nonmax_suppression(gradient_magnitude, gradient_direction)

# 결과 출력
img = Image.fromarray(np.uint8(result), 'L')
img.save('./thin_converted_img.png')